In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import keras
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import preprocess_input

from mobilenet import get_mobilenet

Using TensorFlow backend.


In [3]:
data_dir = '/home/ubuntu/data/'

In [4]:
data_generator = ImageDataGenerator(
    data_format='channels_last',
    preprocessing_function=preprocess_input
)

train_generator = data_generator.flow_from_directory(
    data_dir + 'train', 
    target_size=(224, 224),
    batch_size=64
)

val_generator = data_generator.flow_from_directory(
    data_dir + 'val', 
    target_size=(224, 224),
    batch_size=64
)

Found 25600 images belonging to 256 classes.
Found 5120 images belonging to 256 classes.


In [5]:
model = get_mobilenet()

for layer in model.layers[:-2]:
    layer.trainable = False
    
model.layers[-2].kernel_regularizer = keras.regularizers.l2(1e-4)

In [6]:
model.compile(
    optimizer=optimizers.RMSprop(lr=1e-3), 
    loss='categorical_crossentropy', metrics=['accuracy', 'top_k_categorical_accuracy']
)

In [7]:
model.fit_generator(
    train_generator, 
    steps_per_epoch=150, epochs=5, verbose=1,
    validation_data=val_generator, validation_steps=24, 
    max_queue_size=10, workers=1, use_multiprocessing=False
)

Epoch 1/5
150/150 [==============================] - 46s - loss: 3.2400 - acc: 0.3762 - top_k_categorical_accuracy: 0.5832 - val_loss: 1.7513 - val_acc: 0.6100 - val_top_k_categorical_accuracy: 0.8359
Epoch 2/5
150/150 [==============================] - 46s - loss: 1.2683 - acc: 0.7135 - top_k_categorical_accuracy: 0.8865 - val_loss: 1.1585 - val_acc: 0.7227 - val_top_k_categorical_accuracy: 0.9030
Epoch 3/5
150/150 [==============================] - 46s - loss: 0.8068 - acc: 0.8051 - top_k_categorical_accuracy: 0.9361 - val_loss: 1.0547 - val_acc: 0.7350 - val_top_k_categorical_accuracy: 0.9076
Epoch 4/5
150/150 [==============================] - 46s - loss: 0.5395 - acc: 0.8684 - top_k_categorical_accuracy: 0.9632 - val_loss: 1.0692 - val_acc: 0.7344 - val_top_k_categorical_accuracy: 0.9082
Epoch 5/5
150/150 [==============================] - 46s - loss: 0.5191 - acc: 0.8724 - top_k_categorical_accuracy: 0.9643 - val_loss: 0.9747 - val_acc: 0.7539 - val_top_k_categorical_accuracy: 0.

In [8]:
val_generator_no_shuffle = data_generator.flow_from_directory(
    data_dir + 'val', 
    target_size=(224, 224),
    batch_size=64, shuffle=False
)

Found 5120 images belonging to 256 classes.


In [9]:
model.evaluate_generator(val_generator_no_shuffle, 80)

[0.98461559917777774, 0.75527343749999998, 0.919921875]